In [1]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.9/178.9 KB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 5.9 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=111801ecb65787ab497d02127bc9f261a6ad83dd9f1cca30813ee3a5b32ace1c
  Stored in directory: /root/.cache/pip/wheels/4c/8e/7e/72fbc243e1aeecae64a96875432e70d4e92f3d2d18123be004
Successfully built pathtools
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
pip install shap

     -------------------------------------- 435.6/435.6 kB 9.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import wandb
from wandb.keras import WandbCallback
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
sweep_configuration = {
    'method': 'bayes',
    'name': 'tcn_aguro',
    'metric': {
        'goal': 'minimize', 
        'name': 'validation_loss'
		},
    'parameters': {
        'batch_size': {'value': 512},
        'epochs': {'value': 500},
        'lr': {'max': 0.1, 'min': 0.0001},
        'dropout_rate': {'values': [0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]},
        'optimizer': {'value': 'adam'},
        'nb_filters': {'values': [16, 32, 64]},
        'kernel_size': {'values': [3, 6, 8, 12]}
     }
}

In [6]:
from google.colab import drive
drive.mount('/content/drive')

!pip install pandas==1.4.4

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 96.8 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5


In [2]:
import pandas as pd
# from tsai.all import SlidingWindowPanel
import numpy as np

from sklearn.model_selection import train_test_split

from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import load_model
import tensorflow.keras as keras
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [5]:
def norm_df(df,cols):
  for col in cols:
    df[col] = df[col] / df[col].max()
  return df

In [6]:
df_shift7 = pd.read_pickle('/content/drive/MyDrive/Master/DSP/merge_df_shift_7.pickle')
df_shift7 = df_shift7[['date', 'NDVI_mean', 'NDMI_mean', 'NBSI_mean', 'name', 'tavg',  'prcp',
       'NDVI_mean_shift', 'tavg_shift', 'prcp_shift']]
df_shift14 = pd.read_pickle('/content/drive/MyDrive/Master/DSP/merge_df_shift_14.pickle')
df_shift14 = df_shift14[['date', 'NDVI_mean', 'NDMI_mean', 'NBSI_mean', 'name', 'tavg',
        'prcp','NDVI_mean_shift', 'tavg_shift', 'prcp_shift']]

In [7]:
df_shift7 = norm_df(df_shift7, ['tavg','prcp','tavg_shift','prcp_shift'])
df_shift14 = norm_df(df_shift14, ['tavg','prcp','tavg_shift','prcp_shift'])
df_shift7.dropna(axis = 0, inplace = True)
df_shift14.dropna(axis = 0, inplace = True)

In [ ]:
df_shift14.columns

Index(['date', 'NDVI_mean', 'NDMI_mean', 'NBSI_mean', 'name', 'tavg', 'prcp',
       'NDVI_mean_shift', 'tavg_shift', 'prcp_shift'],
      dtype='object')

In [8]:
x_cols = ['NDVI_mean', 'NDMI_mean', 'NBSI_mean', 'tavg_shift', 'prcp_shift']

In [9]:
sliding_window_X, sliding_window_y = SlidingWindowPanel(window_len=14, unique_id_cols=[
                                                        'name'], get_x=x_cols, get_y=['NDVI_mean_shift'], stride=1, horizon = 14)(df_shift14)


processing data...


...data processed
concatenating X...
...X concatenated
concatenating y...
...y concatenated


In [10]:
sliding_window_y.shape

(48607, 14)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(sliding_window_X, sliding_window_y,
                                                    random_state=42, shuffle=False, test_size=0.2)

In [12]:
X_train.shape

(38885, 5, 14)

In [16]:
X_train = np.moveaxis(X_train, 1, -1)
X_train.shape

(38885, 14, 5)

In [17]:
X_test = np.moveaxis(X_test, 1, -1)
X_test.shape

(9722, 14, 5)

In [ ]:
y_train.shape

(38885, 14)

In [18]:
np.save('X_train14', X_train)

In [19]:
np.save('X_test14', X_test)

In [20]:
np.save('y_train', y_train)
np.save('y_test', y_test)

In [3]:
import tensorflow as tf
from tcn import TCN, compiled_tcn
tf.config.list_physical_devices()


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [5]:
y_train.shape

NameError: name 'y_train' is not defined

In [ ]:
import tensorflow.keras.backend as K
def rmse(y_true,y_pred):
  return K.sqrt(K.mean(K.square(y_true - y_pred)))

In [6]:
def compile_model(swp_X, nb_filters, kernel_size, dilations, dropout_rate, y_train):

    # output tcn2
    nb_features = swp_X.shape[2]
    nb_outputs = y_train.shape[1]

    i = Input(shape=(swp_X.shape[1], nb_features))

    m = TCN(nb_filters=nb_filters, kernel_size=kernel_size, dropout_rate=dropout_rate, dilations=dilations, return_sequences=True, activation='relu')(i)
    m = TCN(nb_filters=nb_filters, dropout_rate=dropout_rate, kernel_size=kernel_size, dilations=dilations, return_sequences=False, activation='relu')(m)
    m = Dense(nb_outputs, activation='linear')(m)
    model = Model(inputs=[i], outputs=[m])

    return model

In [27]:
def get_optimizer(lr=1e-3, optimizer="adam"):
    "Select optmizer between adam and sgd with momentum"
    if optimizer.lower() == "adam":
        return tf.keras.optimizers.Adam(learning_rate=lr)
    if optimizer.lower() == "sgd":
        return tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.1)
    if optimizer.lower() == "adadelta":
        return tf.keras.optimizers.Adadelta(learning_rate=lr)


def train(X_train, y_train, batch_size=512, epochs=500, lr=1e-3, optimizer='adam', dropout_rate=0.1, nb_filters=32, kernel_size=3, dilations=[
            1, 2, 4, 8, 16, 32, 64]):  

    model = compile_model(X_train, nb_filters, kernel_size, dilations, dropout_rate, y_train)
    
    # Compile model like you usually do.
    tf.keras.backend.clear_session()
    model.compile(loss=root_mean_squared_error, 
                  optimizer=get_optimizer(lr, optimizer), 
                  metrics=['mae', root_mean_squared_error])

    # callback setup
    wandb_callbacks = [
        WandbCallback(),
        EarlyStopping(patience=15),
        ReduceLROnPlateau(patience=10)
        #WandbModelCheckpoint(filepath="my_model_{epoch:02d}")
    ]

    model.fit(X_train, 
              y_train, 
              batch_size=batch_size, 
              epochs=epochs, 
              validation_split=0.2, 
              callbacks=wandb_callbacks)

In [28]:
def sweep_train(config_defaults=None):
    # Initialize wandb with a sample project name
    with wandb.init(config=config_defaults, reinit=True):  # this gets over-written in the Sweep

        # Specify the other hyperparameters to the configuration, if any
        wandb.config.architecture_name = "TCN"
        wandb.config.dataset_name = "Sentinel_2_ndvi"

        train(X_train,
              y_train,
              wandb.config.batch_size, 
              wandb.config.epochs,
              wandb.config.lr,
              wandb.config.optimizer,
              wandb.config.dropout_rate,
              wandb.config.nb_filters,
              wandb.config.kernel_size
              )

In [7]:
X_train = np.load('X_train14.npy')
X_test = np.load('X_test14.npy')
y_train = np.load('y_train14.npy')
y_test = np.load('y_test14.npy')

In [9]:
model = compile_model(X_train, 16, 12, [1, 2, 4, 8, 16, 32, 64], 0.6, y_train)

In [10]:
model.compile(loss=root_mean_squared_error,
              optimizer='adam', metrics=['mae', root_mean_squared_error])

In [26]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 14, 5)]           0         
                                                                 
 tcn (TCN)                   (None, 14, 64)            161984    
                                                                 
 tcn_1 (TCN)                 (None, 64)                172928    
                                                                 
 dense (Dense)               (None, 14)                910       
                                                                 
Total params: 335,822
Trainable params: 335,822
Non-trainable params: 0
_________________________________________________________________


In [24]:
X_train = np.load('X_train14.npy')
X_test = np.load('X_test14.npy')
y_train = np.load('y_train14.npy')
y_test = np.load('y_test14.npy')

In [11]:
wandb_callbacks = [
    EarlyStopping(patience=15),
    ReduceLROnPlateau(patience=10)
]

model.fit(X_train, 
          y_train, 
          batch_size=200, 
          epochs=200, 
          validation_split=0.2, 
          callbacks=wandb_callbacks)

Epoch 1/200


: 

: 

In [16]:
sweep_id = wandb.sweep(sweep_configuration, project="aguro_tcn")

Create sweep with ID: 9vw7gd6y
Sweep URL: https://wandb.ai/dsp_aguro/aguro_tcn/sweeps/9vw7gd6y


In [ ]:
wandb.agent(sweep_id, function=sweep_train, count=100)

wandb: Agent Starting Run: oag9879x with config:
wandb: 	batch_size: 512
wandb: 	dropout_rate: 0.5
wandb: 	epochs: 500
wandb: 	kernel_size: 12
wandb: 	lr: 0.0717458955462187
wandb: 	nb_filters: 16
wandb: 	optimizer: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/500
 6/61 [=>............................] - ETA: 4s - loss: 0.9195 - mae: 0.7334 - root_mean_squared_error: 0.9195

61/61 [==============================] - ETA: 0s - loss: 0.3031 - mae: 0.2348 - root_mean_squared_error: 0.3028

wandb: Adding directory to artifact (/content/wandb/run-20230124_115339-oag9879x/files/model-best)... Done. 0.0s


...

61/61 [==============================] - 16s 260ms/step - loss: 0.2103 - mae: 0.1581 - root_mean_squared_error: 0.2103 - val_loss: 0.1651 - val_mae: 0.1418 - val_root_mean_squared_error: 0.1668 - lr: 0.0561
Epoch 3/500
61/61 [==============================] - 6s 97ms/step - loss: 0.2074 - mae: 0.1555 - root_mean_squared_error: 0.2074 - val_loss: 0.1717 - val_mae: 0.1481 - val_root_mean_squared_error: 0.1733 - lr: 0.0561
Epoch 4/500
61/61 [==============================] - ETA: 0s - loss: 0.2051 - mae: 0.1543 - root_mean_squared_error: 0.2050

wandb: Adding directory to artifact (/content/wandb/run-20230124_140416-hda0z343/files/model-best)... Done. 0.0s


61/61 [==============================] - 16s 259ms/step - loss: 0.2051 - mae: 0.1543 - root_mean_squared_error: 0.2050 - val_loss: 0.1578 - val_mae: 0.1289 - val_root_mean_squared_error: 0.1585 - lr: 0.0561
Epoch 5/500
61/61 [==============================] - 6s 98ms/step - loss: 0.2011 - mae: 0.1520 - root_mean_squared_error: 0.2011 - val_loss: 0.1598 - val_mae: 0.1337 - val_root_mean_squared_error: 0.1612 - lr: 0.0561
Epoch 6/500
61/61 [==============================] - 6s 95ms/step - loss: 0.2135 - mae: 0.1568 - root_mean_squared_error: 0.2135 - val_loss: 0.1752 - val_mae: 0.1464 - val_root_mean_squared_error: 0.1756 - lr: 0.0561
Epoch 7/500
61/61 [==============================] - 6s 98ms/step - loss: 0.2007 - mae: 0.1503 - root_mean_squared_error: 0.2007 - val_loss: 0.1646 - val_mae: 0.1387 - val_root_mean_squared_error: 0.1659 - lr: 0.0561
Epoch 8/500
61/61 [==============================] - 6s 97ms/step - loss: 0.1987 - mae: 0.1489 - root_mean_squared_error: 0.1987 - val_loss: 0

wandb: Adding directory to artifact (/content/wandb/run-20230124_140416-hda0z343/files/model-best)... Done. 0.0s


61/61 [==============================] - 16s 263ms/step - loss: 0.2006 - mae: 0.1506 - root_mean_squared_error: 0.2006 - val_loss: 0.1541 - val_mae: 0.1257 - val_root_mean_squared_error: 0.1549 - lr: 0.0561
Epoch 10/500
61/61 [==============================] - 6s 96ms/step - loss: 0.2002 - mae: 0.1496 - root_mean_squared_error: 0.2002 - val_loss: 0.1691 - val_mae: 0.1448 - val_root_mean_squared_error: 0.1708 - lr: 0.0561
Epoch 11/500
61/61 [==============================] - 6s 98ms/step - loss: 0.1983 - mae: 0.1496 - root_mean_squared_error: 0.1983 - val_loss: 0.1672 - val_mae: 0.1380 - val_root_mean_squared_error: 0.1679 - lr: 0.0561
Epoch 12/500
61/61 [==============================] - 6s 93ms/step - loss: 0.1981 - mae: 0.1499 - root_mean_squared_error: 0.1981 - val_loss: 0.1616 - val_mae: 0.1340 - val_root_mean_squared_error: 0.1623 - lr: 0.0561
Epoch 13/500
61/61 [==============================] - 6s 97ms/step - loss: 0.2153 - mae: 0.1591 - root_mean_squared_error: 0.2153 - val_los

In [29]:
model.evaluate(X_test, y_test)

304/304 [==============================] - 2s 6ms/step - loss: 0.1350 - mae: 0.1168 - root_mean_squared_error: 0.1350


[0.13501565158367157, 0.11678452789783478, 0.1349869966506958]

In [30]:
preds = model.predict(X_test)

304/304 [==============================] - 2s 5ms/step


In [31]:
import matplotlib.pyplot as plt

In [13]:
import pandas as pd
import numpy as np

In [14]:
data = pd.read_pickle('df_shift_14.pickle')

In [14]:
x_cols = ['NDVI_mean', 'NDMI_mean', 'NBSI_mean', 'tavg_shift', 'prcp_shift']
start_date = '2018-01-01'
end_date = '2018-1-13'
name = 'almelo'
data.query('date >= @start_date & date <= @end_date & name == @name')



,date,NDVI_mean,NDMI_mean,NBSI_mean,name,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun,NDVI_mean_shift,tavg_shift,prcp_shift
353,2018-01-01,0.843367,-0.329698,0.086237,almelo,6.9,5.1,9.500000,5.600000,NaN,220.0,20.100000,NaN,998.099976,NaN,0.871004,3.8,5.1
354,2018-01-02,0.843614,-0.327495,0.084031,almelo,5.6,4.5,6.200000,0.100000,NaN,224.0,17.700001,NaN,1006.000000,NaN,0.875392,4.2,4.0
355,2018-01-03,0.843992,-0.325926,0.082242,almelo,7.5,6.0,10.200000,25.799999,NaN,237.0,33.500000,NaN,989.099976,NaN,0.879915,1.9,8.3
356,2018-01-04,0.844529,-0.325022,0.080951,almelo,7.3,6.5,9.700000,7.200000,NaN,221.0,22.700001,NaN,993.299988,NaN,0.884516,4.0,5.5
357,2018-01-05,0.845250,-0.324809,0.080240,almelo,6.6,4.7,8.200000,0.400000,NaN,216.0,19.200001,NaN,995.299988,NaN,0.889141,2.5,2.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621,2018-09-26,0.820756,-0.189689,0.043746,almelo,11.7,4.7,18.700001,0.000000,NaN,207.0,13.900000,36.0,1033.000000,NaN,0.830400,14.2,0.0
622,2018-09-27,0.829104,-0.199533,0.050052,almelo,14.6,8.0,21.500000,0.000000,NaN,205.0,8.800000,29.6,1026.900024,NaN,0.804702,17.1,0.0
623,2018-09-28,0.836393,-0.207864,0.055605,almelo,11.1,4.5,15.600000,0.000000,NaN,4.0,11.100000,36.0,1025.800049,NaN,0.760963,19.0,0.0
624,2018-09-29,0.842175,-0.214186,0.059992,almelo,7.1,0.4,15.600000,0.000000,NaN,350.0,2.600000,18.0,1028.800049,NaN,0.737400,20.9,0.0


In [11]:
def get_data(data_set, location, date_range):
    start_date = date_range[0]
    end_date = date_range[-1]
    data_set = data_set.query('date >= @start_date & date <= @end_date & name == @location')
    data_set = data_set[['NDVI_mean', 'NDMI_mean', 'NBSI_mean', 'tavg_shift', 'prcp_shift']]
    return data_set    

In [15]:
x_test = get_data(data, 'almelo', ['2018-01-01', '2018-01-14']).to_numpy()

In [3]:
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [5]:
from tensorflow import keras
from tensorflow.keras import backend as K
from tcn import TCN


model = keras.models.load_model(
    r'tcn_tf\content\tcn_nvdi.tf', custom_objects={'root_mean_squared_error': root_mean_squared_error})


In [39]:
x_test[np.newaxis,...]

(1, 14, 5)

In [16]:
model.predict(x_test[np.newaxis,...])

: 

: 